# Portable cooling system

A portable cooling system uses canisters of volume 2 L charged with refrigerant R134a at 20°C and quality 0.05 (i.e., vapor mass fraction). Heat is transfered from a person to the canister. Saturated vapor escapes from the canister when the pressure in the canister reaches 30°C, controlled by a relief valve. The cooling system stops working and must be discarded when it is empty of liquid refrigerant.

**Problem:** Determine the cooling density of the system: the ratio of the amount of energy that can be absorbed before the canister is discarded to the initial system mass. Compare this to the cooling density of an ice pack (with latent heat of fusion of $\Delta h_{\text{fus}} = 333.6$ J/g).

In [1]:
# load necessary modules

# Numpy adds some useful numerical types and functions
import numpy as np

# Cantera will handle thermodynamic properties
import cantera as ct

# Pint gives us some helpful unit conversion
from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity

Specify initial state:

In [7]:
volume = Q_(2, 'liter')
temp_charge = Q_(20, 'degC')
quality_charge = 0.05

initial = ct.Hfc134a()
initial.TX = temp_charge.to('K').magnitude, quality_charge

Final state, based on the temperature when the relief valve opens and that the fluid is saturated vapor:

In [9]:
final = ct.Hfc134a()
temp_final = Q_(30, 'degC')
quality_final = 1.0
final.TX = temp_final.to('K').magnitude, quality_final

Do a mass balance on the system, over the process from state 1 to state 2:
\begin{equation}
0 = m_{\text{out}} + m_2 - m_1 \;,
\end{equation}
which we can use to find the mass that leaves the canister ($m_{\text{out}}$):

In [11]:
mass_initial = volume / Q_(initial.v, 'm^3/kg')
mass_final = volume / Q_(final.v, 'm^3/kg')
mass_out = mass_initial - mass_final
print(f'mass that left the canister: {mass_out.to(ureg.kg): .2f}')

mass that left the canister: 0.70 kilogram


Next, we can perform an energy balance of the system over the process:
\begin{equation}
Q_{\text{in}} =  m_{\text{out}}  h_{\text{out}} + m_2 u_2 - m_1 u_1 \;,
\end{equation}
where $h_{\text{out}}$ is the enthalpy of the refigerant leaving the system, which has the same properties as the final state in the tank (no liquid, and 30°C):

In [12]:
heat_in = (
    mass_out*Q_(final.h, 'J/kg') + 
    mass_final*Q_(final.u, 'J/kg') - 
    mass_initial*Q_(initial.u, 'J/kg')
    )

THe cooling density (neglecting the canister mass) is
\begin{equation}
CD = \frac{Q_{\text{in}}}{m_1}
\end{equation}

In [15]:
cooling_density = heat_in / mass_initial
cooling_density.ito('W hr/kg')
print(f'cooling density: {cooling_density: .2f}')

cooling_density_ice = Q_(333.6, 'J/g')
cooling_density_ice.ito('W hr/kg')
print(f'cooling density of ice: {cooling_density_ice: .2f}')

cooling density: 49.37 hour * watt / kilogram
cooling density of ice: 92.67 hour * watt / kilogram
